In [145]:
import sleap_track_cleaning
from importlib import reload

reload(sleap_track_cleaning)

<module 'sleap_track_cleaning' from '/Users/emily/code/collaborative-intelligence/elm_central_park_birds/sleap_track_cleaning.py'>

In [146]:
folder_from_colab = '/Users/emily/data/FLIRcam/ForTracking/use_me/from_colab'
h5_files = [f for f in os.listdir(folder_from_colab) if f.endswith('analysis.h5')]

# iterate over .h5 files in folder
for file in h5_files:
    filename = os.path.join(folder_from_colab,file)
    sleap_track_cleaning.generate_cleaned_analysis_file(filename, max_jump = 30, merge_max_distance = 10, merge_max_time = 100, min_length = 10, smooth_win = 9, smooth_poly = 3)

Pre-cleaning, total of 98 tracks
removing jumps resulted in 258 tracks
interpolated missing points
merging in 111 tracks
merging in 34 tracks
merging in 8 tracks
merging in 1 tracks
merging in 0 tracks
interpolated missing points
removing 40 tracks, out of 104
smoothed tracks
Total of 64 tracks
Saved a total of 64 tracks
File saved as /Users/emily/data/FLIRcam/ForTracking/use_me/from_colab/20221215122046189_-5_25_bone.avi.000_20221215122046189_-5_25_bone.analysis_cleaned.h5
Pre-cleaning, total of 82 tracks
removing jumps resulted in 121 tracks
interpolated missing points
merging in 50 tracks
merging in 19 tracks
merging in 3 tracks
merging in 1 tracks
merging in 0 tracks
interpolated missing points
removing 20 tracks, out of 48
smoothed tracks
Total of 28 tracks
Saved a total of 28 tracks
File saved as /Users/emily/data/FLIRcam/ForTracking/use_me/from_colab/20221215121330884_n5_25_bone.avi.000_20221215121330884_n5_25_bone.analysis_cleaned.h5
Pre-cleaning, total of 351 tracks
removing j

In [63]:
import h5py
def print_fields_recursive(group, indent=0):
    """Recursively print datasets and groups in an h5py group."""
    items = sorted(group.items())
    for name, item in items:
        if isinstance(item, h5py.Dataset):  # Check if item is a dataset
            print("  " * indent + f"Dataset: {name} {item.shape}")
        elif isinstance(item, h5py.Group):  # Check if item is a group
            print("  " * indent + f"Group: {name}")
            print_fields_recursive(item, indent + 1)

# filename = 'path_to_your_file.h5'
with h5py.File(filename, 'r') as f:
    print_fields_recursive(f)
with h5py.File(filename, 'r') as f:
    instance_scores = f["track_occupancy"][:]


Dataset: edge_inds (2, 2)
Dataset: edge_names (2, 2)
Dataset: instance_scores (392, 10407)
Dataset: labels_path ()
Dataset: node_names (3,)
Dataset: point_scores (392, 3, 10407)
Dataset: provenance ()
Dataset: track_names (392,)
Dataset: track_occupancy (10407, 392)
Dataset: tracking_scores (392, 10407)
Dataset: tracks (392, 2, 3, 10407)
Dataset: video_ind ()
Dataset: video_path ()


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)